In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
root_dir = os.path.dirname(os.path.dirname(os.getcwd()))
data_dir = os.path.join(root_dir, 'data')
train_path = os.path.join(data_dir, 'train_split.csv')
valid_path = os.path.join(data_dir, 'valid_split.csv')
full_dataset_path = os.path.join(data_dir, 'gx_merged_months.csv')
lags_features_path = os.path.join(data_dir, 'gx_merged_lags_months.csv')
volume_path = os.path.join(data_dir, 'gx_volume.csv')

In [ ]:
full = pd.read_csv(full_dataset_path)
train = pd.read_csv(train_path)
validation = pd.read_csv(valid_path)
test = full.loc[full.test == 1, :]
lags_features = pd.read_csv(lags_features_path)
volume = pd.read_csv(volume_path, index_col=0)

In [ ]:
volume_at_1 = volume.loc[volume.month_num == -1, ['country', 'brand', 'volume']].\
            drop_duplicates().\
            rename(columns={'volume':'volume_1'})

In [ ]:
lags_features = full.merge(volume_at_1, 'left', on=['country', 'brand'])

In [ ]:
lags_features['relative_volume'] = np.log((1 + lags_features.volume)/lags_features.volume_1)
lags_features_train_validation = lags_features.loc[lags_features.test == 0, :]
lags_features_test = lags_features.loc[lags_features.test == 1, :]

In [ ]:
train_with_features = train.merge(lags_features_train_validation, 'inner', on=['country', 'brand'])
validation_with_features = validation.merge(lags_features_train_validation, 'inner', on=['country', 'brand'])

In [ ]:
train_serie = train_with_features.groupby(['month_num'], as_index=False).relative_volume.mean().rename(
    columns={'relative_volume': 'relative_volume_train'})
validation_serie = validation_with_features.groupby(['month_num'], as_index=False).relative_volume.mean().rename(
    columns={'relative_volume': 'relative_volume_val'})
both_series = train_serie.merge(validation_serie, 'inner', on = 'month_num')

In [ ]:
both_series.set_index('month_num').plot(figsize=(10,5), grid=True)